<a href="https://colab.research.google.com/github/akinzayn/NMSUCovidPrevalenceProject/blob/main/Qualtrics_One_drive_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Please Read**

Before usage, make sure to 

1. download the current file from one drive

2. store it in the same folder as this R/Rmd/ipynb file 

3. It is need for making sure that the structure of the excel file that will be generated by this code matches what is expected. The code fails to produce an excel file if that structure has been modified and the code has not been modified to match.

#### **Setting Up Colab**

In [ ]:
#@title
install.packages("pacman") 
library(pacman) # package for computing sensitivity and specificity

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
#@title
h = p_load(tidyverse, googledrive, magrittr, readxl, qualtRics, openxlsx)
# h = p_load(Microsoft365R)

In [ ]:
drive_deauth()
integrity_files = drive_get(as_id(c("")))
h = 1:dim(integrity_files)[1] %>% map(~drive_download(integrity_files[.,], overwrite = TRUE))

#### **Generating Excel file**

##### **Downloading Data From Qualtrics**

In [ ]:
qualtrics_api_credentials(api_key = "", 
                          base_url = "sjc1.qualtrics.com",
                          install = TRUE, overwrite = TRUE)

Your original .Renviron will be backed up and stored in your R HOME directory if needed.

Your Qualtrics key and base URL have been stored in your .Renviron.  
To use now, restart R or run `readRenviron("~/.Renviron")`



In [ ]:
readRenviron("~/.Renviron")

In [ ]:
surveys <- all_surveys() 

In [ ]:
head(surveys, 2)

id,name,ownerId,lastModified,creationDate,isActive
<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
SV_24YyVO3ywHx3Agu,Prevalence of Depression & Sleep Disorders in New Mexico State University Population during COVID-19,UR_0uZUwCWAtCSaKAC,2022-04-25T23:00:36Z,2022-04-01T05:21:24Z,TRUE


In [ ]:

survey_response_df <- fetch_survey(surveyID = surveys$id[1], 
                         verbose = TRUE,
                        #  strip_html = TRUE,
                          import_id = TRUE,
                           time_zone = "America/Denver",
                            add_column_map = TRUE,
                             add_var_labels = TRUE,
                             force_request=TRUE, col_types = cols())                           
head(survey_response_df)

  |======================================================================| 100%


startDate,endDate,status,progress,duration,finished,recordedDate,_recordId,distributionChannel,userLanguage,⋯,QID72_3,QID72_6,QID72_4,QID73_1,QID73_2,QID73_3,QID73_5,QID74,QID75,QID77_TEXT
<dttm>,<dttm>,<chr>,<dbl>,<dbl>,<lgl>,<dttm>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2022-04-22 12:28:08,2022-04-22 12:51:57,IP Address,100,1429,TRUE,2022-04-22 12:51:58,R_1FfW01qiIDYgg3j,anonymous,EN,⋯,NA,NA,NA,Very often (5 or more times per month),NA,NA,NA,No,Yes,hawkinsr@nmsu.edu
2022-04-22 14:48:36,2022-04-22 14:55:41,IP Address,100,425,TRUE,2022-04-22 14:55:42,R_2f2TG06j0f54tl2,anonymous,EN,⋯,9 hours or more per night,NA,NA,Very often (5 or more times per month),NA,NA,NA,Prefer not to respond,Yes,kakakari@nmsu.edu
2022-04-25 10:55:50,2022-04-25 10:56:09,IP Address,100,18,TRUE,2022-04-25 10:56:09,R_3QMOg1Lq7xmYyni,anonymous,EN,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2022-04-25 10:53:26,2022-04-25 10:58:23,IP Address,100,296,TRUE,2022-04-25 10:58:23,R_1mEOoQySNmk7TJi,anonymous,EN,⋯,9 hours or more per night,NA,NA,Very often (5 or more times per month),NA,NA,NA,No,Yes,eeturley@nmsu.edu
2022-04-25 10:52:59,2022-04-25 10:58:23,IP Address,100,323,TRUE,2022-04-25 10:58:23,R_3oZLB1J3wd9V3x5,anonymous,EN,⋯,NA,NA,NA,NA,Occasionally (less than times per month),NA,NA,No,Yes,glparikh@nmsu.edu
2022-04-25 10:53:36,2022-04-25 10:58:37,IP Address,100,300,TRUE,2022-04-25 10:58:38,R_2zpafNIdkYROJ6G,anonymous,EN,⋯,NA,NA,NA,Very often (5 or more times per month),NA,NA,NA,No,Yes,spesto@nmsu.edu


In [ ]:
# survey_question_df <- survey_questions(surveys$id[1])
# head(survey_question_df, 2)

##### **Cleaning Data Pulled From Quatrics API**

In [ ]:
demography_df = survey_response_df %>%
transmute(DateTime = str_split(recordedDate, " ", simplify = TRUE), Date = DateTime[ , 1],
Time = DateTime[ , 2], DateTime = NULL,
Q1 = paste( QID1_1, ' ', QID1_2, ' ', QID1_3, ' ',QID1_4, ' ', QID1_5, ' ', QID1_4_TEXT), Q1 = gsub("NA", "", Q1), Q1 = trimws(gsub("\\s+", " ", Q1)), 
Q2 = QID2,
Q3 = QID3,
Q4 = QID4,
Q5 = paste( QID5_1, ' ', QID5_2, ' ', QID5_3, ' ',QID5_4, ' ', QID5_5, ' ', QID5_6, ' ', QID5_7, ' ', QID5_8, ' ', QID5_7_TEXT), Q5 = gsub("NA", "", Q5), Q5 = trimws(gsub("\\s+", " ", Q5)),
Q6.1 = QID8,
Q6.2 = QID10,
Q6.3 = QID9,
Q6.4 = paste( QID11, ' ', QID11_5_TEXT), Q6.4 = gsub("NA", "", Q6.4), Q6.4 = trimws(gsub("\\s+", " ", Q6.4)), 
Q6.5 = QID15_TEXT,
Q6.6 = QID16,
Q6.7 = QID17,
Q6.8 = paste( QID18_1, ' ', QID18_2, ' ', QID18_3, ' ',QID18_4, ' ', QID18_5, ' ', QID18_6), Q6.8 = gsub("NA", "", Q6.8), Q6.8 = trimws(gsub("\\s+", " ", Q6.8)), 
Q7 = QID14,
Q8 = QID19,
Q9 = QID20,
Q10 = QID21,
Q11 = QID22,
Q12 = QID23,
Q12.1 = paste( QID24_1, ' ', QID24_2, ' ', QID24_3, ' ',QID24_4, ' ', QID24_5, ' ', QID24_6), Q12.1 = gsub("NA", "", Q12.1), Q12.1 = trimws(gsub("\\s+", " ", Q12.1)), 
Q12.2 = QID25,
Q13 = QID27,
Q13.1 = QID28,
Q13.2 = paste( QID29_1, ' ', QID29_2, ' ', QID29_3, ' ',QID29_4, ' ', QID29_5, ' ', QID29_6, ' ', QID29_7), Q13.2 = gsub("NA", "", Q13.2), Q13.2 = trimws(gsub("\\s+", " ", Q13.2)), 
Q13.3 = QID30,
Q13.4 = QID31_TEXT,
Q13.5 = QID32_TEXT,
Q13.6 = QID33,
Q14 = QID34_TEXT,
Q15 = QID35_TEXT,
Q16 = QID36,
Q16.1 = QID37,
Q17 = QID38,
Q18 = QID39,
Q19 = paste( QID40_1, ' ', QID40_2, ' ', QID40_3, ' ',QID40_4, ' ', QID40_5, ' ', QID40_6, ' ', QID40_5_TEXT), Q19 = gsub("NA", "", Q19), Q19 = trimws(gsub("\\s+", " ", Q19)), 
Q20 = paste( QID41_1, ' ', QID41_2, ' ', QID41_3, ' ',QID41_4, ' ', QID41_5, ' ', QID41_6, ' ', QID41_7, ' ', QID41_6_TEXT), Q20 = gsub("NA", "", Q20), Q20 = trimws(gsub("\\s+", " ", Q20)), 
Q21.1 = QID42,
Q21.2 = QID43,
Q21.3 = QID44,
Q21.4 = QID45,
Q21.5 =  paste( QID46, ' ', QID46_7_TEXT), Q21.5 = gsub("NA", "", Q21.5), Q21.5 = trimws(gsub("\\s+", " ", Q21.5)),
Q21.6 = paste( QID47, ' ', QID47_4_TEXT), Q21.6 = gsub("NA", "", Q21.6), Q21.6 = trimws(gsub("\\s+", " ", Q21.6)),
Q21.7 = paste( QID48, ' ', QID47_4_TEXT), Q21.7 = gsub("NA", "", Q21.7), Q21.7 = trimws(gsub("\\s+", " ", Q21.7)),
Q21.8 = paste( QID49, ' ', QID49_1_TEXT), Q21.8 = gsub("NA", "", Q21.8), Q21.8 = trimws(gsub("\\s+", " ", Q21.8)),
Q22 = QID50,
Q22.1 = paste( QID51, ' ', QID51_1_TEXT,' ', QID51_4_TEXT), Q22.1 = gsub("NA", "", Q22.1), Q22.1 = trimws(gsub("\\s+", " ", Q22.1)),
Q22.2 = QID52_TEXT


)

In [ ]:
head(demography_df)

Date,Time,Q1,Q2,Q3,Q4,Q5,Q6.1,Q6.2,Q6.3,⋯,Q21.2,Q21.3,Q21.4,Q21.5,Q21.6,Q21.7,Q21.8,Q22,Q22.1,Q22.2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2022-04-22,12:51:58,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,No,⋯,No,No,No,No,No,No,No,Yes,,NA
2022-04-22,14:55:42,Student Faculty,Female,20-24,Single,Non-Hispanic White,No,Yes,"Yes, I am a student worker",⋯,No,No,No,No,No,No,No,Yes,,NA
2022-04-25,10:56:09,,NA,NA,NA,,NA,NA,NA,⋯,NA,NA,NA,,,,,NA,,NA
2022-04-25,10:58:23,Student,Female,18-19,Single,Mexican-American,No,Yes,No,⋯,No,No,No,No,No,No,No,Yes,,NA
2022-04-25,10:58:23,Faculty Staff,Female,30-34,Married,Non-Hispanic White Asian,NA,No,NA,⋯,No,No,No,No,No,No,No,Yes,,NA
2022-04-25,10:58:38,Student,Female,20-24,Single,Non-Hispanic White Asian,No,Yes,No,⋯,No,No,No,No,No,No,No,Yes,,NA


In [ ]:
covid_19_df = survey_response_df %>%
transmute(
Q1 = QID53,
Q2 = QID54,
Q3 = QID55,
Q4.1 = QID56,
Q4.2 = QID57,
Q4.3 = QID58,
Q4.4 = QID59,
Q4.5 = QID118,
Q5 = paste( QID60, ' ', QID60_1_TEXT), Q5 = gsub("NA", "", Q5), Q5 = trimws(gsub("\\s+", " ", Q5)), 
Q6 = QID61,
Q6.1 = QID115,
Q7.1 = QID62,
Q7.2 = QID63,
Q7.3 = QID64,
Q7.4 = QID65,
Q7.5 = QID99,
Q8 = paste( QID66, ' ', QID66_1_TEXT), Q5 = gsub("NA", "", Q5), Q5 = trimws(gsub("\\s+", " ", Q5)), 
Q9 = QID67,
Q10 = QID68_TEXT

)

head(covid_19_df)

Q1,Q2,Q3,Q4.1,Q4.2,Q4.3,Q4.4,Q4.5,Q5,Q6,Q6.1,Q7.1,Q7.2,Q7.3,Q7.4,Q7.5,Q8,Q9,Q10
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
No,No,No,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond,Vaccinated,2 doses,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond,Prefer not to respond NA,Yes,"Work travel, depression"
Yes,Yes,No,No,No,No,No,Yes,"Yes (Please specify [i.e. parent, sibling, etc.]) great grandparent",Vaccinated,3 doses,"Yes, all living parents are vaccinated","Yes, all living siblings are vaccinated",Prefer not to respond,Prefer not to respond,"Yes, all living grandparents are vaccinated",No NA,No,fear of being unable to work due to quarantine but not receive any form of financial compensation
NA,NA,NA,NA,NA,NA,NA,NA,,NA,NA,NA,NA,NA,NA,NA,NA NA,NA,NA
No,Yes,No,No,Yes,No,No,No,No,Vaccinated,2 doses,"No, not all living parents are vaccinated","Yes, all living siblings are vaccinated",No,"No, not all are vaccinated","Yes, all living grandparents are vaccinated",No NA,No,NA
No,No,No,Yes,Yes,Yes,No,Yes,No,Vaccinated,2 doses,"Yes, all living parents are vaccinated","Yes, all living siblings are vaccinated",Yes,Prefer not to respond,"Yes, all living grandparents are vaccinated",No NA,No,Getting a new position after completing PhD
No,No,No,No,No,No,No,Yes,No,Vaccinated,3 doses,"Yes, all living parents are vaccinated","Yes, all living siblings are vaccinated",Prefer not to respond,"Yes, all are vaccinated","Yes, all living grandparents are vaccinated",No NA,No,NA


In [ ]:
depresssion_df = survey_response_df %>%
transmute(
Q1 = QID102,
Q2 = QID103,
Q3 = QID104,
Q4 = QID105,
Q5 = QID106,
Q6 = QID107,
Q7 = QID108,
Q8 = QID109,
Q9 = QID110

)

head(depresssion_df)

Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,2,2,3,2,3,3,1,1
2,3,3,3,1,1,3,1,0
NA,NA,NA,NA,NA,NA,NA,NA,NA
3,3,3,3,3,3,3,3,1
2,1,2,2,1,2,2,0,0
2,3,2,3,2,3,1,2,2


In [ ]:
sleep_df = survey_response_df %>%
transmute(
Q1 = QID71,
Q2 = paste( QID72_1, ' ', QID72_2, ' ', QID72_3, ' ',QID72_4), Q2 = gsub("NA", "", Q2), Q2 = trimws(gsub("\\s+", " ", Q2)), 
Q3 = paste( QID73_1, ' ', QID73_2, ' ', QID73_3, ' ',QID73_5), Q3 = gsub("NA", "", Q3), Q3 = trimws(gsub("\\s+", " ", Q3)), 
Q4 = QID74,
Compensation = QID75,
Email = QID77_TEXT
)

head(sleep_df)

Q1,Q2,Q3,Q4,Compensation,Email
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month),No,Yes,hawkinsr@nmsu.edu
"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night 9 hours or more per night,Very often (5 or more times per month),Prefer not to respond,Yes,kakakari@nmsu.edu
NA,,,NA,NA,NA
"Yes, sleeping longer each night",9 hours or more per night,Very often (5 or more times per month),No,Yes,eeturley@nmsu.edu
"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night,Occasionally (less than times per month),No,Yes,glparikh@nmsu.edu
"Sleeping about the same amount each night, but sleep is disturbed",6 hours or less per night,Very often (5 or more times per month),No,Yes,spesto@nmsu.edu


In [ ]:
data_df = cbind(demography_df, covid_19_df, depresssion_df, sleep_df)

In [ ]:
head(data_df)

,Date,Time,Q1,Q2,Q3,Q4,Q5,Q6.1,Q6.2,Q6.3,⋯,Q6,Q7,Q8,Q9,Q1,Q2,Q3,Q4,Compensation,Email
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2022-04-22,12:51:58,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,No,⋯,3,3,1,1,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month),No,Yes,hawkinsr@nmsu.edu
2,2022-04-22,14:55:42,Student Faculty,Female,20-24,Single,Non-Hispanic White,No,Yes,"Yes, I am a student worker",⋯,1,3,1,0,"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night 9 hours or more per night,Very often (5 or more times per month),Prefer not to respond,Yes,kakakari@nmsu.edu
3,2022-04-25,10:56:09,,NA,NA,NA,,NA,NA,NA,⋯,NA,NA,NA,NA,NA,,,NA,NA,NA
4,2022-04-25,10:58:23,Student,Female,18-19,Single,Mexican-American,No,Yes,No,⋯,3,3,3,1,"Yes, sleeping longer each night",9 hours or more per night,Very often (5 or more times per month),No,Yes,eeturley@nmsu.edu
5,2022-04-25,10:58:23,Faculty Staff,Female,30-34,Married,Non-Hispanic White Asian,NA,No,NA,⋯,2,2,0,0,"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night,Occasionally (less than times per month),No,Yes,glparikh@nmsu.edu
6,2022-04-25,10:58:38,Student,Female,20-24,Single,Non-Hispanic White Asian,No,Yes,No,⋯,3,1,2,2,"Sleeping about the same amount each night, but sleep is disturbed",6 hours or less per night,Very often (5 or more times per month),No,Yes,spesto@nmsu.edu


In [ ]:
dim(data_df)

[1] 428  83

##### **Automation Integrity**

***Read the template file***

In [ ]:
# get file that is used in setting up this automation
suppressMessages({integrity_df = read_excel("Covid_Integrity_Apr_23_5_09_Am.xlsx", col_types = "text")})
head(integrity_df)

22XXX = Undergraduate students,...2,...3,...4,55XXX = Graduate students,...6,...7,77XXX = Faculty,...9,99XXX = Staff,⋯,...75,...76,...77,...78,...79,...80,...81,...82,...83,...84
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,NA,I: Demographics,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,IV: Sleep Disorder Questionnaire,NA,NA,NA,NA,NA
ResponseNo,Date,Time,Q1,Q2,Q3,Q4,Q.5,Q6.1,Q6.2,⋯,Q6,Q7,Q8,Q9,Q1,Q2,Q3,Q4,Compensation,Email
22001.0,44673.0,12:52pm,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,⋯,3.0,3.0,1.0,1.0,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month),No,Yes,hawkinsr@nmsu.edu
22002.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
22003.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
22004.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
integrity_vec = integrity_df[2,] %>% t()
rownames(integrity_vec) = NULL


***Upload the current file downloaded from Onedrive before running cell***

In [ ]:
# get latest file downloaded from one_drive
suppressMessages({one_drive_df = read_excel("Prevalence of Depression and Sleep Disorders in New Mexico State University Populations during COVID-19 - Response Tracker.xlsx", col_types = "text")})
head(one_drive_df)

22XXX = Undergraduate students,...2,...3,...4,55XXX = Graduate students,...6,...7,77XXX = Faculty,...9,99XXX = Staff,⋯,...75,...76,...77,...78,...79,...80,...81,...82,...83,...84
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
NA,NA,NA,I: Demographics,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,IV: Sleep Disorder Questionnaire,NA,NA,NA,NA,NA
ResponseNo,Date,Time,Q1,Q2,Q3,Q4,Q.5,Q6.1,Q6.2,⋯,Q6,Q7,Q8,Q9,Q1,Q2,Q3,Q4,Compensation,Email
22001,44673,12:52pm,Student,Male,20-24,Single,Non-Hispanic White,No,Yes,⋯,3,3,1,1,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month),No,Yes,hawkinsr@nmsu.edu
22002,44673,2:55pm,Student Faculty,Female,20-24,Single,Non-Hispanic White,No,Yes,⋯,1,3,1,0,"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night 9 hours or more per night,Very often (5 or more times per month) NA NA NA,Prefer not to respond,Yes,kakakari@nmsu.edu
22003,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
22004,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
# fixing colum names
one_drive_vec = one_drive_df[2,] %>% t()
rownames(one_drive_vec) = NULL


In [ ]:
# check dimension

if(sum(integrity_vec != one_drive_vec) == 0){
}else{stop("Automation Fails Integrity Test: Check that the Template file of Googledrive matches the Template file on One drive")}

if(sum(names(one_drive_df) != names(integrity_df)) == 0){
}else{stop("Automation Fails Integrity Test: Check that the Template file of Googledrive matches the Template file on One drive")}

NULL

NULL

In [ ]:
colnames(one_drive_df) = one_drive_vec
# head(one_drive_df, 2)
colnames(integrity_df) = integrity_vec
# head(one_drive_df, 2)

##### **Filtering Into Undergrad, Grad, Staff and Faculty Data**

In [ ]:
suppressMessages({one_drive_df_name_fixed = tibble(one_drive_df, .name_repair = "universal")})

In [ ]:
suppressMessages({data_df_name_fixed = tibble(data_df, .name_repair = "universal")})

data_df_name_fixed = data_df_name_fixed %>% mutate(Num_row = 1:n()) %>% dplyr::select(Num_row, everything())

In [ ]:

empty_data_df_name_fixed = data_df_name_fixed %>% filter(if_all("Time":"Email", is.na))

In [ ]:
empty_data_df_name_fixed

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


Num_row,Date,Time,Q1...3,Q2...4,Q3...5,Q4...6,Q5...7,Q6.1...8,Q6.2,⋯,Q6...74,Q7...75,Q8...76,Q9...77,Q1...78,Q2...79,Q3...80,Q4...81,Compensation,Email
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [ ]:
# filter groups

# Student only categories

undergrad_student_only_df = data_df_name_fixed %>% filter(grepl("^Student$", Q1...3) & grepl("Yes", Q6.2))
# head(undergrad_student_only_df,2)
grad_student_only_df = data_df_name_fixed %>% filter(grepl("^Student$", Q1...3) & grepl("Yes", Q6.1...8) )
# head(grad_student_only_df,2)
simply_student_only  = data_df_name_fixed %>% filter(grepl("^Student$", Q1...3) & !grepl("Yes", Q6.1...8) & !grepl("Yes", Q6.2))
# head(simply_student_only,2) 
simply_student_only_with_errors  = data_df_name_fixed %>% filter(grepl("^Student$", Q1...3) & (grepl("^Prefer not to respond$", Q6.1...8) | grepl("^Prefer not to respond$", Q6.2)) )
# head(simply_student_only_with_errors,2)

# Student intersecting faculty category

undergrad_student_faculty_df = data_df_name_fixed %>% filter(grepl("Student", Q1...3) & grepl("Faculty", Q1...3) & grepl("Yes", Q6.2))
# head(undergrad_student_faculty_df,2)
grad_student_faculty_df = data_df_name_fixed %>% filter(grepl("Student", Q1...3) & grepl("Faculty", Q1...3) & grepl("Yes", Q6.1...8) )
# head(grad_student_faculty_df,2)
simply_student_faculty  = data_df_name_fixed %>% filter(grepl("Student", Q1...3) & grepl("Faculty", Q1...3)  & !grepl("Yes", Q6.1...8) & !grepl("Yes", Q6.2))
# head(simply_student_faculty,2) 
simply_student_faculty_with_errors  = data_df_name_fixed %>% filter(grepl("Student", Q1...3) & grepl("Faculty", Q1...3) & (grepl("^Prefer not to respond$", Q6.1...8) | grepl("^Prefer not to respond$", Q6.2)) )
# head(simply_student_faculty_with_errors,2)

# Student intersecting staff category

undergrad_student_staff_df = data_df_name_fixed %>% filter(grepl("Student", Q1...3) & grepl("Staff", Q1...3) & grepl("Yes", Q6.2))
# head(undergrad_student_staff_df,2)
grad_student_staff_df = data_df_name_fixed %>% filter(grepl("Student", Q1...3) & grepl("Staff", Q1...3) & grepl("Yes", Q6.1...8) )
# head(grad_student_staff_df,2)
simply_student_staff  = data_df_name_fixed %>% filter(grepl("Student", Q1...3) & grepl("Staff", Q1...3)  & !grepl("Yes", Q6.1...8) & !grepl("Yes", Q6.2))
# head(simply_student_staff,2) 
simply_student_staff_with_errors  = data_df_name_fixed %>% filter(grepl("Student", Q1...3) & grepl("Staff", Q1...3) & (grepl("^Prefer not to respond$", Q6.1...8) | grepl("^Prefer not to respond$", Q6.2)) )
# head(simply_student_staff_with_errors,2)

# Faculty only 

faculty_only_df = data_df_name_fixed  %>% filter(grepl("^Faculty$", Q1...3))
# head(faculty_only_df,2)

# Faculty intersecting staff categories

faculty_staff_only_df = data_df_name_fixed %>% filter(grepl("Staff", Q1...3) & grepl("Faculty", Q1...3))
# head(faculty_staff_only_df,2)

# Staff Only

staff_only_df = data_df_name_fixed  %>% filter(grepl("^Staff$", Q1...3))
# head(staff_only_df,2)

# Prefer not to respond

prefer_not_to_respond_df = data_df_name_fixed %>% filter(grepl("^Prefer not to respond$", Q1...3))
# head(prefer_not_to_respond_df,2)

# Other
other_df = data_df_name_fixed %>% filter(grepl("Other", Q1...3))
# head(other_df,2)

In [ ]:
nrow(other_df) + nrow(prefer_not_to_respond_df) + nrow(staff_only_df) +
 nrow(faculty_staff_only_df) + nrow(faculty_only_df) + nrow(simply_student_staff) + nrow(grad_student_staff_df) +
  nrow(undergrad_student_staff_df) + nrow(simply_student_faculty) + nrow(grad_student_faculty_df)  + nrow(undergrad_student_faculty_df) +
    nrow(simply_student_only) + nrow(grad_student_only_df) + nrow(undergrad_student_only_df) + 
     nrow(simply_student_only_with_errors) + nrow(simply_student_faculty_with_errors) + nrow(simply_student_staff_with_errors) 
  # nrow() + nrow() + nrow() +   nrow() + nrow() + nrow()
nrow(data_df_name_fixed)

[1] 424

[1] 428

In [ ]:

suppressMessages({splitted_df =  bind_rows(other_df , prefer_not_to_respond_df , staff_only_df , 
   faculty_staff_only_df , faculty_only_df , simply_student_staff , grad_student_staff_df , undergrad_student_staff_df , simply_student_faculty , grad_student_faculty_df , undergrad_student_faculty_df ,
     simply_student_only , grad_student_only_df , undergrad_student_only_df , 
    simply_student_only_with_errors , simply_student_faculty_with_errors , simply_student_staff_with_errors) %>% distinct(Num_row, .keep_all = T) })





In [ ]:
nrow(splitted_df)

[1] 423

In [ ]:
nrow(data_df_name_fixed)

[1] 428

In [ ]:
weird_entries_df = data_df_name_fixed[!(data_df_name_fixed$Num_row %in% splitted_df$Num_row),]

In [ ]:
weird_entries_df %>% nrow

weird_entries_df

[1] 5

Num_row,Date,Time,Q1...3,Q2...4,Q3...5,Q4...6,Q5...7,Q6.1...8,Q6.2,⋯,Q6...74,Q7...75,Q8...76,Q9...77,Q1...78,Q2...79,Q3...80,Q4...81,Compensation,Email
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
3,2022-04-25,10:56:09,,NA,NA,NA,,NA,NA,⋯,NA,NA,NA,NA,NA,,,NA,NA,NA
241,2022-04-25,12:04:15,,NA,NA,NA,,NA,NA,⋯,NA,NA,NA,NA,NA,,,NA,NA,NA
255,2022-04-25,12:13:38,,NA,NA,NA,,NA,NA,⋯,NA,NA,NA,NA,NA,,,NA,NA,NA
316,2022-04-25,13:39:25,,NA,NA,NA,,NA,NA,⋯,NA,NA,NA,NA,NA,,,NA,NA,NA
392,2022-04-25,17:01:55,,NA,NA,NA,,NA,NA,⋯,NA,NA,NA,NA,NA,,,NA,NA,NA


In [ ]:
ncol(data_df_name_fixed) - sum(is.na(data_df_name_fixed[3, 4:ncol(data_df_name_fixed)]))

[1] 20

In [ ]:
# weird_entries_df = weird_entries_df %>% select(-Num_row)
# splitted_df = splitted_df  %>% select(-Num_row)

##### **Inserting Response Number**

In [ ]:
add_response_num = function(df, start_num, num_row_add = 20){

df = tryCatch(
{
df %>% mutate(ResponseNo = start_num:(start_num - 1 + n()) ) %>% dplyr::select(ResponseNo, everything())
},
error = function(e){
        rbind(df, rep("", ncol(df))) %>% mutate(ResponseNo = start_num:(start_num - 1 + n()) ) %>% dplyr::select(ResponseNo, everything())
    }
)

df_safe = names(df)

to_add <- data.frame(matrix(ncol = ncol(df), nrow = num_row_add))
 colnames(to_add) = df_safe

to_add$ResponseNo = (start_num - 1 + nrow(df)):((start_num - 2 + nrow(df)) + num_row_add)

df = rbind(df, to_add) 

df = tryCatch({
 df %>% select(-Num_row)
},
error = function(e){
        df    
        }
)
return(df)

}



undergrad_student_combine = suppressMessages({bind_rows( undergrad_student_staff_df ,  undergrad_student_faculty_df , undergrad_student_only_df)  %>%
 distinct(Num_row, .keep_all = T) })

grad_student_combine = suppressMessages({bind_rows( grad_student_staff_df ,  grad_student_faculty_df , grad_student_only_df )  %>%
 distinct(Num_row, .keep_all = T) })
 
simply_student_combine = suppressMessages({bind_rows( simply_student_staff , simply_student_faculty , 
     simply_student_only , simply_student_only_with_errors , 
     simply_student_faculty_with_errors , simply_student_staff_with_errors) %>%
      distinct(Num_row, .keep_all = T) })

faculty_combine = suppressMessages({bind_rows( faculty_staff_only_df , faculty_only_df ) %>% distinct(Num_row, .keep_all = T)}) 


undergrad_student_combine = undergrad_student_combine  %>% add_response_num(start_num = 22001)
# head(undergrad_student_combine)

grad_student_combine = grad_student_combine  %>% add_response_num(start_num = 55001)
# head(grad_student_combine)

faculty_combine = faculty_combine %>% add_response_num(start_num = 77001)
# head(faculty_combine)

staff_only_df = staff_only_df  %>% add_response_num(start_num = 99001)
# head(staff_only_df)

# prefer_not_to_respond_df = prefer_not_to_respond_df  %>% add_response_num(start_num = 110001, num_row_add = 5)
# head(prefer_not_to_respond_df)

other_df = other_df  %>% add_response_num(start_num = 330001, num_row_add = 2)

weird_entries_df = weird_entries_df  %>% add_response_num(start_num = 110001, num_row_add = 5)
# head(weird_entries)

last_df = rbind(staff_only_df, other_df) 
last_df = rbind(last_df, weird_entries_df)
# head(other_df)

In [ ]:
names

function (x)  .Primitive("names")

###### **Find Where to Start Writing**

In [ ]:
one_drive_df_name_fixed$Row_num = 2:(nrow(one_drive_df_name_fixed) + 1) # for match rows with where to start

undergrad_student_start_write = one_drive_df_name_fixed %>% filter(ResponseNo == 22001) %>% select(Row_num)
grad_student_start_write = one_drive_df_name_fixed %>% filter(ResponseNo == 55001) %>% select(Row_num)
faculty_start_write = one_drive_df_name_fixed %>% filter(ResponseNo == 77001) %>% select(Row_num)
staff_start_write = one_drive_df_name_fixed %>% filter(ResponseNo == 99001) %>% select(Row_num)


faculty_start_write
staff_start_write
undergrad_student_start_write
grad_student_start_write

Row_num
<int>
504


Row_num
<int>
754


Row_num
<int>
4


Row_num
<int>
254


In [ ]:
head(undergrad_student_combine)

ResponseNo,Date,Time,Q1...4,Q2...5,Q3...6,Q4...7,Q5...8,Q6.1...9,Q6.2,⋯,Q6...75,Q7...76,Q8...77,Q9...78,Q1...79,Q2...80,Q3...81,Q4...82,Compensation,Email
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
22001,2022-04-25,10:59:50,Student Staff,Male,18-19,Single,Prefer not to respond,No,Yes,⋯,0,0,0,0,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month),No,Yes,jrydecki@nmsu.edu
22002,2022-04-25,11:00:40,Student Staff,Male,25-29,Single,Non-Hispanic White,No,Yes,⋯,2,2,3,9,"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night,Very often (5 or more times per month),No,Yes,sth92@nmsu.edu
22003,2022-04-25,11:01:01,Student Staff,Female,18-19,Single,Mexican-American,No,Yes,⋯,1,3,1,0,"Yes, sleeping longer each night",7 to 8 hours per night 9 hours or more per night,Very often (5 or more times per month),No,Yes,hayleynavarrette@gmail.com
22004,2022-04-25,11:01:16,Student Staff,Female,18-19,Single,Mexican-American,No,Yes,⋯,0,1,0,0,"Yes, not sleeping as long each night",7 to 8 hours per night,Very often (5 or more times per month),No,Yes,mickiesb@nmsu.edu
22005,2022-04-25,11:11:48,Student Staff,Male,30-34,Married,Other Hispanic,No,Yes,⋯,1,3,1,0,"Sleeping about the same amount each night, but sleep is disturbed",7 to 8 hours per night,Very often (5 or more times per month),No,Yes,fresques@nmsu.edu
22006,2022-04-25,11:48:35,Student Staff,Female,20-24,Single,Non-Hispanic White,No,Yes,⋯,1,2,2,1,"Yes, not sleeping as long each night",6 hours or less per night,Very often (5 or more times per month),No,Yes,nsmith2@nmsu.edu


##### **Writing Data To Workbook**

In [ ]:
# Write Dataframe
wb <- loadWorkbook("Prevalence of Depression and Sleep Disorders in New Mexico State University Populations during COVID-19 - Response Tracker.xlsx")

writeData(wb, sheet = "Sheet1", x = undergrad_student_combine, startRow = undergrad_student_start_write, rowNames = FALSE, colNames = FALSE)

writeData(wb, sheet = "Sheet1", x = grad_student_combine, startRow = grad_student_start_write, rowNames = FALSE, colNames = FALSE)

writeData(wb, sheet = "Sheet1", x = faculty_combine, startRow = faculty_start_write, rowNames = FALSE, colNames = FALSE)

writeData(wb, sheet = "Sheet1", x = last_df, startRow = staff_start_write, rowNames = FALSE, colNames = FALSE)

border_col_num = c(3, 50, 69, 78, 82, 84)
# style 
s1 <- createStyle(fontSize = 11, wrapText = TRUE, halign = "center")
s2 <-  createStyle(border = "right")

# # Apply styles to the workbook
addStyle(wb, sheet = "Sheet1", style = s1, stack = TRUE,  rows = 1:(nrow(one_drive_df_name_fixed)+1), cols = 1:(ncol(one_drive_df_name_fixed)+1), gridExpand = TRUE)
addStyle(wb, sheet = "Sheet1", style = s2, stack = TRUE,  rows = 1:(nrow(one_drive_df_name_fixed)+1), cols = border_col_num, gridExpand = TRUE)

saveWorkbook(wb, paste0(Sys.Date()," Test_file (openxlsx)",".xlsx"), overwrite = TRUE)

##### **Pushing To Onedrive**

In [ ]:
# get_personal_onedrive(auth_type="device_code")

# get_business_onedrive(tenant="nmsu.edu", auth_type="device_code")

In [ ]:
# read_url_csv <- function(url, ...){
#   tmpFile <- tempfile()
#   download.file(url, destfile = tmpFile)
#   url_csv <- readr::read_csv(tmpFile, ...)
#   return(url_csv)
# }
# onedrive_url <- "https://eltnmsu-my.sharepoint.com/:x:/g/personal/jonaggie_nmsu_edu/ETrw5EUUJW1OkfoqgknUWSMB9mgomEJ6xf7qyite3k4U5A"

# csv <- read_url_csv(onedrive_url)



##### **Convert IPYNB file to R and Rmd**

In [ ]:
colab_files = drive_get(as_id(c("1T4ex861d_lIcyx04-zGhz7Lc4o93CbIu")))
h = 1:dim(integrity_files)[1] %>% map(~drive_download(colab_files[.,], overwrite = TRUE))

In [ ]:
rmarkdown:::convert_ipynb("Qualtrics_One_drive_Automation.ipynb")

In [ ]:
knitr::purl("Qualtrics_One_drive_Automation.Rmd")



processing file: Qualtrics_One_drive_Automation.Rmd



  |......................................................................| 100%


output file: Qualtrics_One_drive_Automation.R




[1] "Qualtrics_One_drive_Automation.R"